<a href="https://colab.research.google.com/github/omaskii/Complete-Python-3-Bootcamp/blob/master/MovieLens_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLens Case Study
Author: **Solomon Akatakpo**

### INTRODUCTION

The **key questions** that needs to be answered with the help of this analysis are listed below:
1. What is the trend of the number of movies released after 1990?
2. How many movies belong to a particular genre?
3. Which movies have more than one genre?
4. Which of the genres are most liked by the users?
5. Which of the movies have been most rated by the users?
6. What are the top 10 movies that have received most rating counts?
7. Is there any relationship between demographic details of the users and ratings for the movies?

1. Connect to google drive and the folder that contains the data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# import libraries
import pandas as pd
import numpy as np
import datetime as dt


#Load datasets from folder location in google drive

In [9]:
movie = pd.read_csv('/content/drive/MyDrive/GreatLearning/movie.csv')
rating = pd.read_csv('/content/drive/MyDrive/GreatLearning/rating.csv')
user = pd.read_csv('/content/drive/MyDrive/GreatLearning/user.csv')

In [10]:
#To display the first five rows of the user dataset
user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [11]:
# see the shape of the user dataset

print('Number of rows and columns in user dataset is: ',user.shape)

Number of rows and columns in user dataset is:  (943, 5)


In [12]:
#checking the data types of the columns to ensure that the data is stored in the correct format
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user id     943 non-null    int64 
 1   age         943 non-null    int64 
 2   gender      943 non-null    object
 3   occupation  943 non-null    object
 4   zip code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


In [13]:
#This shows that user id and age columns are integer while the other columns are object data type

In [14]:
# statistical description of the user dataset
user.describe()

,user id,age
count,943.000000,943.000000
mean,472.000000,34.051962
std,272.364951,12.192740
min,1.000000,7.000000
25%,236.500000,25.000000
50%,472.000000,31.000000
75%,707.500000,43.000000
max,943.000000,73.000000


user age range from 7 years to 73 years for people who rated the movies. From the age column, the average age of user who rated the movie is 34. Statistical summary of the user id column is not useful here since it is an unique identifier

In [15]:
#To display the first five rows of the movie dataset
movie.head()

,movie id,movie title,release date,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,01-01-1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye,01-01-1995,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms,01-01-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty,01-01-1995,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat,01-01-1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
#to get the shape of the movie dataset
print('Number of rows and columns in movie dataset is: ', movie.shape)

Number of rows and columns in movie dataset is:  (1680, 21)


movie id, movie title and release date are related to the movies, and the other columns are the genres of the movies

In [17]:
#To check the data types of the columns contained in the movie dataset
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie id      1680 non-null   int64 
 1   movie title   1680 non-null   object
 2   release date  1680 non-null   object
 3   Action        1680 non-null   int64 
 4   Adventure     1680 non-null   int64 
 5   Animation     1680 non-null   int64 
 6   Childrens     1680 non-null   int64 
 7   Comedy        1680 non-null   int64 
 8   Crime         1680 non-null   int64 
 9   Documentary   1680 non-null   int64 
 10  Drama         1680 non-null   int64 
 11  Fantasy       1680 non-null   int64 
 12  Film-Noir     1680 non-null   int64 
 13  Horror        1680 non-null   int64 
 14  Musical       1680 non-null   int64 
 15  Mystery       1680 non-null   int64 
 16  Romance       1680 non-null   int64 
 17  Sci-Fi        1680 non-null   int64 
 18  Thriller      1680 non-null   int64 
 19  War   

All the columns in the movie dataset are integer except the movie title and release date which are object data type

In [18]:
#The 'release date' need to be changed from object to 'date-time' so that we can use if for further computation
movie['release_date'] = pd.to_datetime(movie['release date'], format = "%d-%m-%Y")
# dropping the old 'release date' column
movie.drop('release date',axis=1,inplace = True)


In [19]:
#Statistical summary of the movie dataset
movie.describe()

,movie id,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000
mean,841.525595,0.149405,0.080357,0.025000,0.072619,0.300595,0.064881,0.029762,0.431548,0.013095,0.014286,0.054762,0.033333,0.036310,0.147024,0.060119,0.149405,0.042262,0.016071
std,485.609591,0.356593,0.271926,0.156171,0.259587,0.458653,0.246389,0.169980,0.495440,0.113717,0.118701,0.227583,0.179559,0.187115,0.354235,0.237778,0.356593,0.201246,0.125788
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.250000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


The statistical summary does not give meaningful summary of the dataset

In [20]:
movie['release_date'].describe(datetime_is_numeric = True) #Since the date is non linear, we use 'date_is_numeric' = argument

count                             1680
mean     1989-08-04 13:10:17.142857088
min                1922-01-01 00:00:00
25%                1993-01-01 00:00:00
50%                1995-01-01 00:00:00
75%                1996-10-18 00:00:00
max                1998-10-23 00:00:00
Name: release_date, dtype: object

The above summary shows that movie release date range from 1922 - 1988

**Ratings Dataset**

Display the first 5 rows of the rating dataset


In [21]:
rating.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


The rating dataset contain rating information given by a user for a particular movie 

In [22]:
#Get the shape of the rating dataset
rating.shape

(100000, 4)

In [23]:
#alternate way to print this out is
print('Number of rows and columns in rating dataset is: ', rating.shape)

Number of rows and columns in rating dataset is:  (100000, 4)


In [24]:
#To check the data type of the columns in the rating dataset 
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user id    100000 non-null  int64
 1   movie id   100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


Rating contain data type - integer, since they are all numbers

To get the statistical summary of the ratings


In [25]:
rating.describe()

,user id,movie id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


This above summary shows that the ratings range from minimum of 1 to maximum of 5. The average rating is 3.53

# **What is the total number of movies released each year after 1990?**

In [26]:
#first extract the year from the release date using datetime.year method
movie['year'] = movie['release_date'].dt.year

In [27]:
#Now to calculate the total number of movie released each year after 1990
movie[movie['year']>1990]['year'].value_counts()

1996    355
1997    286
1995    220
1994    214
1993    126
1998     65
1992     37
1991     22
Name: year, dtype: int64

**Highlights**
The obseravtion made here is that the total number of movies released each year increases except for 1997 and 1998. Top three years with highest number of movies released are '1996', '1997' and '1995'

# **How many movies belong to a particular genre**

In [28]:
#here we need to use the movie dataset since this dataset contain the movie genre

In [29]:
#print out all the columns in the movie dataset
movie.columns

Index(['movie id', 'movie title', 'Action', 'Adventure', 'Animation',
       'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western', 'release_date', 'year'],
      dtype='object')

In [30]:
#To check ythe number of columns in the dataset
len(movie.columns)

22

In [31]:
#The best approach is to use slicing to create a list that contains genre only
genres = movie.columns[2:20]
#we now use the len()function to calculate the number of genres
len(genres)

18

There are 18 genres in the movie dataset

In [32]:
genres #prints the genre list

Index(['Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [33]:
movie[genres].sum() #finds the sum of each genre column

Action         251
Adventure      135
Animation       42
Childrens      122
Comedy         505
Crime          109
Documentary     50
Drama          725
Fantasy         22
Film-Noir       24
Horror          92
Musical         56
Mystery         61
Romance        247
Sci-Fi         101
Thriller       251
War             71
Western         27
dtype: int64

Actiongenre has 251 movies, Adventure genre has 135 and so on. Would be very useful to sort in descending order to see the trend

In [34]:
movie[genres].sum().sort_values(ascending = False)

Drama          725
Comedy         505
Action         251
Thriller       251
Romance        247
Adventure      135
Childrens      122
Crime          109
Sci-Fi         101
Horror          92
War             71
Mystery         61
Musical         56
Documentary     50
Animation       42
Western         27
Film-Noir       24
Fantasy         22
dtype: int64

Movies are distributed across a total of 18 genres
Drama genre has the highest number of movies i.e. 725 in total. This is followed by Comedy genre with 505. Third on the list is Action genre with 251 movies

# **Which movie has more than one genre?**

The basic approach is to create a new column and find the total number of genres for that row. The next step will be to find the number of movies that has more than 1 genre

In [35]:
movie["Genre Total"] = movie.loc[:, genres].sum(axis=1)

In [36]:
movie.head()

,movie id,movie title,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_date,year,Genre Total
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1995-01-01,1995,3
1,2,GoldenEye,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995-01-01,1995,3
2,3,Four Rooms,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995-01-01,1995,1
3,4,Get Shorty,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1995-01-01,1995,3
4,5,Copycat,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1995-01-01,1995,3


A new column, 'Genre Total' has been created

In [37]:
#To pick up the movies with Genre Total >1
movie[movie['Genre Total']>1]

,movie id,movie title,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release_date,year,Genre Total
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1995-01-01,1995,3
1,2,GoldenEye,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995-01-01,1995,3
3,4,Get Shorty,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1995-01-01,1995,3
4,5,Copycat,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1995-01-01,1995,3
6,7,Twelve Monkeys,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1995-01-01,1995,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,1669,MURDER and murder,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,1997-06-20,1997,3
1667,1670,Tainted,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1998-02-01,1998,2
1670,1673,Mirage,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995-01-01,1995,2
1676,1679,B. Monkey,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1998-02-06,1998,2


In [38]:
movie[movie['Genre Total']>1].shape

(849, 23)

In [39]:
movie.shape

(1680, 23)

In [40]:
#Calculate the percentage of movies with >1 genre
perc = (movie[movie['Genre Total']>1].shape[0]/movie.shape[0])*100
print('The percentage of movies having more than one genre is ', round(perc,2),'%')

The percentage of movies having more than one genre is  50.54 %


There are 849 movies with more than one genre out of the original dataset with 1680 movies. i.e. 831 movies with single genre
Percentage of movies having more than one genre is 50.54 %.

**What genre is most liked by users**?

The movie dataset containing the genre information and the rating dataset containing the rating information will be joined using the column that is common to both which is the 'movie id' column.




In [41]:
movie_rating_merge = movie.merge(rating, on = 'movie id', how = 'inner')
movie_rating_merge.head()

,movie id,movie title,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Sci-Fi,Thriller,War,Western,release_date,year,Genre Total,user id,rating,timestamp
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,308,4,887736532
1,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,287,5,875334088
2,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,148,4,877019411
3,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,280,4,891700426
4,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,66,3,883601324


Find the average rating and compare among them

In [42]:
genres

Index(['Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

The approach here is to run a loop across the merged dataframe for the columns that are present in the genres list. Whenever a particular genre is found to have  '1' value in its column, the rating corresponding to that '1' value will be extracted and replaced. All others will be replaced with np.nan
nan stands for not a number - undefined values

In [43]:
# To create an empty dataframe
mean_rating = pd.DataFrame()

# Iterating through the columns of df_merge to extract and store the ratings for each genre
for i in genres:
  mean_rating[i] = np.where(movie_rating_merge[i]==1,movie_rating_merge['rating'],np.nan)

# printing the newly created dataframe
mean_rating.head()

,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Calculating the mean of each column and sorting them in ascending order
mean_rating.mean(axis=0).sort_values(ascending = False)

Film-Noir      3.921523
War            3.815812
Drama          3.687379
Documentary    3.672823
Mystery        3.638132
Crime          3.632278
Romance        3.621705
Western        3.613269
Animation      3.576699
Sci-Fi         3.560723
Musical        3.521397
Thriller       3.509007
Adventure      3.503527
Action         3.480245
Comedy         3.394073
Childrens      3.353244
Horror         3.290389
Fantasy        3.215237
dtype: float64

'Film-Noir' genre is having the highest average ratings with an average rating of 3.92 followed by 'War' genre with rating of 3.81. So, the majority of the population tends to like the 'Film-Noir' and 'War' movies compared to any other movies genres.
Although, the total number of movies in 'Film-Noir' and 'Fantasy' are almost equal, 'Fantasy' genre is having the lowest average ratings with an average rating of 3.21.
Among 18 genres, only 5 genres have received a rating below 3.5 which means around 72% of genres are rated above average

**What is the most preferred movie by users?**

The movie and ratings tables have the information to answer this question. Both datasets can be merged on the basis of a common variable to get the average rating for all the movies. These tables have been merged above so it can be called again

In [45]:
movie_rating_merge.head()

,movie id,movie title,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Sci-Fi,Thriller,War,Western,release_date,year,Genre Total,user id,rating,timestamp
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,308,4,887736532
1,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,287,5,875334088
2,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,148,4,877019411
3,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,280,4,891700426
4,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,1995-01-01,1995,3,66,3,883601324


In order to find the movies preferred by users we need to first get average ratings of each movie using the group_by function



In [47]:
# reset_index will reset the index of dataframe to default indexing (0 to number of rows minus 1)
avrg_rating = movie_rating_merge.groupby(['movie title']).mean()[['rating']].reset_index() 

avrg_rating

,movie title,rating
0,'Til There Was You,2.333333
1,1-900,2.600000
2,101 Dalmatians,2.908257
3,12 Angry Men,4.344000
4,187,3.024390
...,...,...
1652,Young Guns,3.207921
1653,Young Guns II,2.772727
1654,"Young Poisoner's Handbook, The",3.341463
1655,Zeus and Roxanne,2.166667


In [48]:
avrg_rating.rename(columns={'rating':'Avrg_rating'},inplace=True)
avrg_rating.head() #renames the rating column to average rating

,movie title,Avrg_rating
0,'Til There Was You,2.333333
1,1-900,2.600000
2,101 Dalmatians,2.908257
3,12 Angry Men,4.344000
4,187,3.024390


In [49]:
#To get the movies with the highest rating
avrg_rating.sort_values(ascending=False, by= 'Avrg_rating')

,movie title,Avrg_rating
631,"Great Day in Harlem, A",5.0
1170,Prefontaine,5.0
1354,Someone Else's America,5.0
461,Entertaining Angels: The Dorothy Day Story,5.0
942,Marlene Dietrich: Shadow and Light (,5.0
...,...,...
111,Babyfever,1.0
834,Lashou shentan,1.0
1307,Shadows (Cienie),1.0
1304,Shadow of Angels (Schatten der Engel),1.0


In [50]:
avrg_rating[avrg_rating['Avrg_rating']==5] #subset the data with the given condition

,movie title,Avrg_rating
30,Aiqing wansui,5.0
461,Entertaining Angels: The Dorothy Day Story,5.0
631,"Great Day in Harlem, A",5.0
942,Marlene Dietrich: Shadow and Light (,5.0
1170,Prefontaine,5.0
1270,"Saint of Fort Washington, The",5.0
1274,Santa with Muscles,5.0
1354,Someone Else's America,5.0
1382,Star Kid,5.0
1466,They Made Me a Criminal,5.0


There are 10 movies that are highly rated i.e. rating of 5. Theses are the movies that are most liked by users. Other movies that that have rating of 1.0 are the least liked by users. There are movies rated by user more than once which may also affect the overall rating

What are the top 25 movies that have received most rating counts along with their mean rating

In [52]:
count_movie= movie_rating_merge.groupby(['movie title'])['rating'].count().reset_index()
count_movie

,movie title,rating
0,'Til There Was You,9
1,1-900,5
2,101 Dalmatians,109
3,12 Angry Men,125
4,187,41
...,...,...
1652,Young Guns,101
1653,Young Guns II,44
1654,"Young Poisoner's Handbook, The",41
1655,Zeus and Roxanne,6


In [53]:
count_movie.rename(columns={'rating':'Rating_counts'},inplace=True)
count_movie.head() #renames the rating column to Rating_counts

,movie title,Rating_counts
0,'Til There Was You,9
1,1-900,5
2,101 Dalmatians,109
3,12 Angry Men,125
4,187,41


In [54]:
count_movie.sort_values(ascending=False, by= 'Rating_counts')

,movie title,Rating_counts
1393,Star Wars,583
332,Contact,509
497,Fargo,508
1232,Return of the Jedi,507
858,Liar Liar,485
...,...,...
1085,"Object of My Affection, The",1
1088,Office Killer,1
1109,"Other Voices, Other Rooms",1
1123,Paris Was a Woman,1


Star Wars has been rated the most number of times, and few movies likearis Was a Woman ,Á köldum klaka (Cold Fever) have been rated only once 

There are possibly other movies in the list which have been rated more than say 100 times

In [56]:
movie_rated_100 = count_movie[count_movie['Rating_counts']>100] #this will subset the dataset 'movie_count' as per the specified condition
movie_rated_100

,movie title,Rating_counts
2,101 Dalmatians,109
3,12 Angry Men,125
7,2001: A Space Odyssey,259
15,Absolute Power,127
16,"Abyss, The",151
...,...,...
1627,Willy Wonka and the Chocolate Factory,326
1638,"Wizard of Oz, The",246
1646,"Wrong Trousers, The",118
1651,Young Frankenstein,200


The data shows that there are 334 movies that are rated more than 100 times by the users

From th list of movies we can find the number of movies that are preferred mostly by users

In [57]:
avrg_rating.head()

,movie title,Avrg_rating
0,'Til There Was You,2.333333
1,1-900,2.600000
2,101 Dalmatians,2.908257
3,12 Angry Men,4.344000
4,187,3.024390


In [58]:
movie_rated_100.head()

,movie title,Rating_counts
2,101 Dalmatians,109
3,12 Angry Men,125
7,2001: A Space Odyssey,259
15,Absolute Power,127
16,"Abyss, The",151


In [59]:
#merge the above two tables to have a combined view on the basis of average rating and rating costs
#the movie title column is common to both so we merge the tables using this column
movies_top= avrg_rating.merge(movie_rated_100, on = 'movie title', how = 'inner')
movies_top

,movie title,Avrg_rating,Rating_counts
0,101 Dalmatians,2.908257,109
1,12 Angry Men,4.344000,125
2,2001: A Space Odyssey,3.969112,259
3,Absolute Power,3.370079,127
4,"Abyss, The",3.589404,151
...,...,...,...
329,Willy Wonka and the Chocolate Factory,3.631902,326
330,"Wizard of Oz, The",4.077236,246
331,"Wrong Trousers, The",4.466102,118
332,Young Frankenstein,3.945000,200


In [61]:
movies_top.sort_values(ascending=False, by='Avrg_rating').head(25) # helps to display top 25 movies on the basis of average rating

,movie title,Avrg_rating,Rating_counts
63,"Close Shave, A",4.491071,112
260,Schindler's List,4.466443,298
331,"Wrong Trousers, The",4.466102,118
54,Casablanca,4.456790,243
269,"Shawshank Redemption, The",4.445230,283
248,Rear Window,4.387560,209
319,"Usual Suspects, The",4.385768,267
291,Star Wars,4.358491,583
1,12 Angry Men,4.344000,125
59,Citizen Kane,4.292929,198


In [62]:
movies_top.sort_values(ascending=False, by='Rating_counts').head(25) #display top 25 movies on the basis of rating counts

,movie title,Avrg_rating,Rating_counts
291,Star Wars,4.358491,583
69,Contact,3.803536,509
107,Fargo,4.155512,508
251,Return of the Jedi,4.007890,507
178,Liar Liar,3.156701,485
99,"English Patient, The",3.656965,481
261,Scream,3.441423,478
309,Toy Story,3.878319,452
8,Air Force One,3.631090,431
159,Independence Day (ID4),3.438228,429


The data contain **334 movies** in total which have received ratings more than 100 times
**Close Shave, A** has maximum average rating with a rating count of 112
**Star Wars** has received maximum number counts with an average rating of 4.35
There are few movies which have least rating counts like **Paris Was a Woman ,Á köldum klaka (Cold Fever)**

Is there any relationship between demographic details of the users and ratings for the movies?

In this case, the entire data will be merged

In [63]:
# Merging user dataset with movie and ratings(already merged : df_merge) dataset
movies_merge_all = movie_rating_merge.merge(user, on = 'user id', how = 'inner')

In [64]:
movies_merge_all.head()

,movie id,movie title,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,release_date,year,Genre Total,user id,rating,timestamp,age,gender,occupation,zip code
0,1,Toy Story,0,0,1,1,1,0,0,0,...,1995-01-01,1995,3,308,4,887736532,60,M,retired,95076
1,4,Get Shorty,1,0,0,0,1,0,0,1,...,1995-01-01,1995,3,308,5,887737890,60,M,retired,95076
2,5,Copycat,0,0,0,0,0,1,0,1,...,1995-01-01,1995,3,308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys,0,0,0,0,0,0,0,1,...,1995-01-01,1995,2,308,4,887738847,60,M,retired,95076
4,8,Babe,0,0,0,1,1,0,0,1,...,1995-01-01,1995,3,308,5,887736696,60,M,retired,95076


Checking the various attributes of the users in order to know the preferences for the movies and their genres

In [65]:
user['gender'].value_counts(normalize=True) #this will give the proportion of males and females users extracted from User dataset 

M    0.710498
F    0.289502
Name: gender, dtype: float64

In [66]:
movies_merge_all.groupby('gender').rating.mean() #grouping the dataset on 'gender' column and taking the average of ratings

gender
F    3.531510
M    3.529333
Name: rating, dtype: float64

The average rating given by Males and Females is almost equal with a value of around 3.53

In [67]:
movies_merge_all.groupby('occupation').rating.mean().sort_values(ascending= False) #grouping the dataset on 'occupation' column and taking the average of ratings.

occupation
none             3.777778
lawyer           3.735316
doctor           3.688889
educator         3.670692
artist           3.653380
administrator    3.635646
scientist        3.611273
salesman         3.582944
programmer       3.568205
librarian        3.560781
other            3.552335
engineer         3.541473
technician       3.532097
student          3.515167
marketing        3.485641
retired          3.466750
entertainment    3.440783
writer           3.376152
executive        3.349794
homemaker        3.301003
healthcare       2.896220
Name: rating, dtype: float64

non working Users have given highest ratings as compared to working professionals
Lowest ratings on an average are from healthcare professionals

In [68]:
movies_merge_all.groupby(['occupation','gender']).rating.mean()

occupation     gender
administrator  F         3.781839
               M         3.555233
artist         F         3.347065
               M         3.875841
doctor         M         3.688889
educator       F         3.699132
               M         3.660246
engineer       F         3.751724
               M         3.537676
entertainment  F         3.448889
               M         3.439807
executive      F         3.773756
               M         3.320340
healthcare     F         2.736021
               M         3.639839
homemaker      F         3.278810
               M         3.500000
lawyer         F         3.623188
               M         3.741379
librarian      F         3.580070
               M         3.537920
marketing      F         3.522624
               M         3.474801
none           F         3.632877
               M         3.876636
other          F         3.531114
               M         3.563447
programmer     F         3.577566
               M         3

Around 71% of users are Males while only 29% users are females
The average rating given by Males and Females is almost equal with a value of around 3.53 and it can be said that the ratings given by Males and Females does not differ significantly.
Users who are not working have given highest ratings as compared to users who are working professionals.
Healthcare workers have given lowest ratings on an average
The low ratings from the healthcare sector is majorly driven by female workers

# **Executive Summary**

The number of movie releases up to 1998 has increased steadily.

There are a total of 18 genres in which movies are distributed.
The top five genres by movie count are Drama, Comedy, Action, Thriller, and Romance.

Of the 1680 movies, 50% have multiple genres while the rest have a single genre.
Film-Noir has the highest average rating at 3.92, while Fantasy has the lowest at 3.21.

Of the 18 genres, 72% have an above-average rating of 3.5 or higher.
Top-rated movies include Great Day in Harlem-A, Prefontaine, while least-rated movies include Shadow of Angels (Schatten der Engel), Power 98, etc.

There are 334 movies rated more than 100 times.

Close Shave-A has the highest average rating of 4.49 with 112 ratings, while Star Wars has the most ratings with 583 and an average rating of 4.35.

Approximately 71% of users are male, while 29% are female.

The average rating given by males and females is around 3.53.

Users who are not working have rated movies higher than working professionals.
Healthcare workers have given the lowest average ratings.